# Kabuto Auto Trader - 日次パフォーマンス分析

このノートブックでは、取引データから日次パフォーマンスを分析します。

## 分析内容
- 日次損益グラフ
- 累積損益カーブ
- 基本統計（勝率、PF、シャープレシオ等）
- ドローダウン分析

In [ ]:
# ライブラリインポート
import sys
sys.path.append('../lib')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from data_loader import KabutoDataLoader, quick_load_trades
from analytics import PerformanceAnalyzer

# 日本語フォント設定（macOS）
plt.rcParams['font.family'] = 'Hiragino Sans'
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_style('whitegrid')

print("ライブラリのインポート完了")

## 1. データ読み込み

ExcelファイルまたはサンプルデータからJupyter Notebookを読み込みます。

In [ ]:
# オプション1: Excelファイルから読み込み（実データ）
# EXCEL_PATH = '../../Kabuto Auto Trader.xlsm'
# loader = KabutoDataLoader(excel_path=EXCEL_PATH)
# trades = loader.load_all_trades(source='excel')

# オプション2: サンプルデータを使用（テスト用）
trades = KabutoDataLoader.generate_sample_data(n_trades=200)

print(f"読み込んだ取引数: {len(trades)}")
print(f"期間: {trades['timestamp'].min()} - {trades['timestamp'].max()}")
trades.head()

## 2. 基本統計

In [ ]:
# PerformanceAnalyzer 初期化
analyzer = PerformanceAnalyzer(trades)

# レポート出力
analyzer.print_report()

## 3. 日次損益グラフ

In [ ]:
# 日次損益集計
trades['date'] = trades['timestamp'].dt.date
daily_pnl = trades.groupby('date')['pnl'].sum()

# グラフ描画
fig, ax = plt.subplots(figsize=(14, 6))
ax.bar(daily_pnl.index, daily_pnl.values, color=['green' if x > 0 else 'red' for x in daily_pnl.values])
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.set_xlabel('日付', fontsize=12)
ax.set_ylabel('日次損益（円）', fontsize=12)
ax.set_title('日次損益推移', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"最高日次損益: {daily_pnl.max():,.0f}円")
print(f"最低日次損益: {daily_pnl.min():,.0f}円")
print(f"平均日次損益: {daily_pnl.mean():,.0f}円")

## 4. 累積損益カーブ

In [ ]:
# 累積損益
trades_sorted = trades.sort_values('timestamp').copy()
trades_sorted['cumulative_pnl'] = trades_sorted['pnl'].cumsum()

# グラフ描画
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(trades_sorted['timestamp'], trades_sorted['cumulative_pnl'], linewidth=2, color='blue')
ax.fill_between(trades_sorted['timestamp'], 0, trades_sorted['cumulative_pnl'], alpha=0.3)
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
ax.set_xlabel('日時', fontsize=12)
ax.set_ylabel('累積損益（円）', fontsize=12)
ax.set_title('累積損益カーブ', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"最終累積損益: {trades_sorted['cumulative_pnl'].iloc[-1]:,.0f}円")

## 5. ドローダウン分析

In [ ]:
# ドローダウン計算
cumulative_pnl = trades_sorted['cumulative_pnl']
running_max = cumulative_pnl.cummax()
drawdown = running_max - cumulative_pnl

# グラフ描画
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# 累積損益 + 最高値
axes[0].plot(trades_sorted['timestamp'], cumulative_pnl, label='累積損益', linewidth=2)
axes[0].plot(trades_sorted['timestamp'], running_max, label='最高値', linestyle='--', linewidth=1, color='orange')
axes[0].set_ylabel('累積損益（円）', fontsize=12)
axes[0].set_title('累積損益 vs 最高値', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# ドローダウン
axes[1].fill_between(trades_sorted['timestamp'], 0, -drawdown, color='red', alpha=0.5)
axes[1].set_xlabel('日時', fontsize=12)
axes[1].set_ylabel('ドローダウン（円）', fontsize=12)
axes[1].set_title('ドローダウン推移', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

dd_stats = analyzer.get_drawdown_stats()
print(f"最大ドローダウン: {dd_stats['max_drawdown']:,.0f}円 ({dd_stats['max_drawdown_pct']:.1f}%)")
print(f"平均ドローダウン: {dd_stats['average_drawdown']:,.0f}円")

## 6. 損益分布

In [ ]:
# ヒストグラム
fig, ax = plt.subplots(figsize=(12, 6))
ax.hist(trades['pnl'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='損益0円')
ax.axvline(x=trades['pnl'].mean(), color='green', linestyle='--', linewidth=2, label=f'平均損益 ({trades["pnl"].mean():,.0f}円)')
ax.set_xlabel('損益（円）', fontsize=12)
ax.set_ylabel('頻度', fontsize=12)
ax.set_title('損益分布', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. 銘柄別パフォーマンス（オプション）

In [ ]:
# 銘柄別統計
ticker_stats = analyzer.get_ticker_stats()

if not ticker_stats.empty:
    print("\n銘柄別パフォーマンス:")
    print(ticker_stats.to_string(index=False))
    
    # グラフ
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(ticker_stats['ticker'], ticker_stats['total_pnl'], color='skyblue', edgecolor='black')
    ax.set_xlabel('銘柄コード', fontsize=12)
    ax.set_ylabel('総損益（円）', fontsize=12)
    ax.set_title('銘柄別総損益', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("銘柄データがありません")

## 8. まとめ

このノートブックでは、日次パフォーマンスを詳細に分析しました。

次のステップ:
- `02_monthly_report.ipynb` で月次レポートを確認
- `03_trade_analysis.ipynb` で個別トレード分析
- `04_backtest_simulator.ipynb` でバックテストシミュレーション